In [1]:
import albumentations as A
from albumentations.augmentations.transforms import Flip

import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [2]:
import os
import dataset
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from efficientnet_pytorch import EfficientNet


class Classifier(pl.LightningModule):

    def __init__(self,args):
        super().__init__()
        self.args = args
        
        #self.resnet = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=False,num_classes = 2)
        self.resnet = EfficientNet.from_pretrained('efficientnet-b3',num_classes = 2)

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        x = self.resnet(x)
        prob = F.softmax(x)
        return x, prob
    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        p, _ = self(x)
        loss = F.cross_entropy(p, y.long())
        # Logging to TensorBoard by default
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        p, _ = self(x)
        
        loss = F.cross_entropy(p, y.long())
        f1 = f1_loss(y,p.argmax(1))
        # Logging to TensorBoard by default
        self.log('val_loss', loss)
        self.log('val_f1',f1)
        return loss
    
    def test_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        
        x, y = batch
        p, _ = self(x)
        
        loss = F.cross_entropy(p, y.long())
        f1 = f1_loss(y,p.argmax(1))
        # Logging to TensorBoard by default
        self.log('test_loss', loss)
        self.log('test_f1',f1)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        return optimizer
    
    def train_dataloader(self):
        train_ds = dataset.getTrainDs(self.args['train_tr'])
        loader= DataLoader(train_ds,batch_size = self.args['batch_size'],num_workers = 6,shuffle=True)
        return loader
    
    def val_dataloader(self):
        val_ds = dataset.getValDs(self.args['val_tr'])
        loader= DataLoader(val_ds,batch_size = self.args['batch_size'],num_workers = 6)
        return loader
    def test_dataloader(self):
        val_ds = dataset.getValDs(self.args['val_tr'])
        loader= DataLoader(val_ds,batch_size = self.args['batch_size'],num_workers = 6)
        return loader
    
    def predict_dataloader(self):
        val_ds = dataset.getTestDs(self.args['val_tr'])
        loader= DataLoader(val_ds,batch_size = self.args['batch_size'],num_workers = 6)
        return loader
    
    
def f1_loss(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:

    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2

    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)


    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)

    epsilon = 1e-7

    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset
import cv2

class ImgDataset(Dataset):
    def __init__(self,df,mode,transforms = None):
        self.imageID = df['ImageID']
        self.labels = df['label']
        self.transforms = transforms
        self.labelmap = {'perseverance':0,'curiosity':1}
        self.mode = mode
        
    def __getitem__(self,x):
        path = self.imageID.iloc[x]
        label = float(self.labelmap[self.labels.iloc[x]])
        i = cv2.imread(f'data/{self.mode}/'+str(path)+'.jpg')
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            i = self.transforms(image = i)['image']
            
        i = torch.tensor(i) / 255.0
        #.unsqueeze_(-1)
        i = i.permute(2,0,1)
        if self.mode != 'test':
            return i, label
        else:
            return i
    
    def __len__(self):
        return len(self.imageID)
    
def getTrainDs(train_tr = None):
    train_df = pd.read_csv('data/train.csv')
    return ImgDataset(train_df,'train',train_tr)

def getValDs(val_tr):
    val_df = pd.read_csv('data/val.csv')
    return ImgDataset(val_df,'val',val_tr)

def getTestDs(test_tr):
    val_df = pd.read_csv('data/sample_submission.csv')
    return ImgDataset(val_df,'test',test_tr)

def writeSub(p):
    test_df = pd.read_csv('data/sample_submission.csv')
    

In [4]:
if __name__ == '__main__':
    trainer = Trainer(max_epochs = 6, gpus = 1, precision=16, amp_level='O1',deterministic=True)
    
    train_tr = A.Compose([
        A.CenterCrop(200,200,always_apply=True),
        Flip()
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(200,200,always_apply=True)
    ])
    
    model = Classifier({'lr':3e-4,'batch_size':64,'train_tr':train_tr,'val_tr':val_tr})
    
    trainer.fit(model)
    trainer.test(model)
    out = trainer.predict(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


Loaded pretrained weights for efficientnet-b3


Testing: 0it [00:00, ?it/s]

<ipython-input-2-9872615ed450>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(x)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1': 0.08072998374700546, 'test_loss': 0.7056330442428589}
--------------------------------------------------------------------------------


Predicting: 0it [00:00, ?it/s]

In [5]:
import pandas as pd
def writeSub(p):
    labelmap = {0:'perseverance',1:'curiosity'}
    test_df = pd.read_csv('D:/rover/ch1/data/sample_submission.csv')
    output_list = p.int().tolist()
    output_list = [labelmap[i] for i in output_list]
    test_df['label'] = output_list
    test_df.to_csv('submission.csv',index = False)

In [6]:
output = torch.tensor([])

for i in range(len(out)):
    output = torch.cat((output,torch.tensor(out[i][1]).argmax(1)))

In [7]:
writeSub(output)